### Riddler Express

Start with 5 highlighted squares on an `infinite` board!

To go from one generation to the next, you consider every square’s eight neighbors (up, down, left, right and the four diagonal directions). If at least three of those squares are shaded, in the previous iteration, that square will be shaded in the next generation.

How many squares will be shaded in generation 10?



In [1]:
import numpy as np

# build initial state (gen 1)
matrix = np.zeros((9,9)) # start with 0s
overwrite = [(4,3), (4,4), (4,5), (3,4), (5,4)]
for idx in overwrite:
    matrix[idx] = 1

print(matrix)

[[0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 1. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [2]:
def searchNeighbors(idx, m) -> int:
    """Pass in index and check if >= 3 neighbors are 1"""
    i,j = idx
    elements = [m[i, j-1], m[i-1, j], m[i+1, j], m[i, j+1],
                m[i-1, j-1], m[i-1, j+1], m[i+1, j-1], m[i+1, j+1]
               ]
    
    # check if v > any surrounding vals
    g = len([e for e in elements if e])
    
    if g >= 3:
        return 1
    return 0

assert(searchNeighbors((4,4), matrix) == 1) #center has 4, so should return a 1
assert(searchNeighbors((3,3), matrix) == 1) #(3,3) was 0, but switches to 1
assert(searchNeighbors((0,0), matrix) == 0) #(0,0) was 0, and stays 0

In [3]:
def fullSearch(m):
    """Pass in matrix, pad with 0s for easier index, and search over all"""
    m_pad = np.pad(m, pad_width=1, mode = 'constant', constant_values = 0)
    row = m.shape[0]
    col = m.shape[1]
    update = []
    for i in range(1,row+1):
        for j in range(1,col+1):
            if searchNeighbors((i,j), m_pad):
                update.append((i,j))
    for idx in update:
        m_pad[idx] = 1
    
    return m_pad

assert(np.sum(fullSearch(matrix)) == 9)

In [4]:
n = 10
for i in range(1,n+1):
    print(f"Total shaded in generation {i}: {np.sum(matrix)}")
    matrix = fullSearch(matrix)

Total shaded in generation 1: 5.0
Total shaded in generation 2: 9.0
Total shaded in generation 3: 13.0
Total shaded in generation 4: 21.0
Total shaded in generation 5: 29.0
Total shaded in generation 6: 37.0
Total shaded in generation 7: 49.0
Total shaded in generation 8: 61.0
Total shaded in generation 9: 73.0
Total shaded in generation 10: 89.0


### Extra Credit:

Extra credit: As N gets very, very large, approximately how many squares will be shaded in generation N (in terms of N)?

Need to rebuild, much too slow now

In [5]:
### rerun first 10 to confirm approach
arr = np.zeros((9,9)) # start with 0s
overwrite = [(4,3), (4,4), (4,5), (3,4), (5,4)]
for idx in overwrite:
    arr[idx] = 1

### gen
gen = 10

for i in range(1,gen+1):
    print(f"Total shaded in generation {i}: {np.sum(arr)}")
    arr = np.pad(arr, pad_width=1, mode = 'constant', constant_values = 0)
    r,c = arr.shape
    empty_arr = np.zeros((r,c))

    # rows
    empty_arr[1:] += arr[:r-1] # same as summing row above (i - 1, j)
    empty_arr[:r-1] += arr[1:r] # same as summing row below (i + 1, j)

    # cols
    empty_arr[:, 1:] += arr[:, :c-1] # same as summing left col (i, j - 1)
    empty_arr[:, :c-1] += arr[:, 1:c] # same as summing right col (i, j + 1)

    # diag down-right
    empty_arr[:r-1, :c-1] += arr[1:r, 1:c] 

    # diag up-right
    empty_arr[1:, :c-1] += arr[:r-1, 1:c] 

    # diag down-left
    empty_arr[:r-1, 1:] += arr[1:r, :c-1] 

    # diag down-right
    empty_arr[1:, 1:] += arr[:r-1, :c-1] 

    # rewrite as 1 where >= 3, else 0 -> overwrite
    arr = np.where(empty_arr >= 3 , 1, 0)

Total shaded in generation 1: 5.0
Total shaded in generation 2: 9
Total shaded in generation 3: 13
Total shaded in generation 4: 21
Total shaded in generation 5: 29
Total shaded in generation 6: 37
Total shaded in generation 7: 49
Total shaded in generation 8: 61
Total shaded in generation 9: 73
Total shaded in generation 10: 89


In [6]:
### rerun first 10
import time
arr = np.zeros((9,9)) # start with 0s
overwrite = [(4,3), (4,4), (4,5), (3,4), (5,4)]
for idx in overwrite:
    arr[idx] = 1

### gen
gen = 2_000
start = time.time()

### store output:
sum_n = []

for i in range(1,gen+1):
    if i % 400 == 0:
        print(f"Total shaded in generation {i}: {np.sum(arr)}")
        print(f"Total time: {time.time() - start:.2}")
    
    # add sum relative to n
    sum_n.append(np.sum(arr))
        
    # pad and solve
    arr = np.pad(arr, pad_width=1, mode = 'constant', constant_values = 0)
    r,c = arr.shape
    empty_arr = np.zeros((r,c))

    # rows
    empty_arr[1:] += arr[:r-1] # same as summing row above (i - 1, j)
    empty_arr[:r-1] += arr[1:r] # same as summing row below (i + 1, j)

    # cols
    empty_arr[:, 1:] += arr[:, :c-1] # same as summing left col (i, j - 1)
    empty_arr[:, :c-1] += arr[:, 1:c] # same as summing right col (i, j + 1)

    # diag down-right
    empty_arr[:r-1, :c-1] += arr[1:r, 1:c] 

    # diag up-right
    empty_arr[1:, :c-1] += arr[:r-1, 1:c] 

    # diag down-left
    empty_arr[:r-1, 1:] += arr[1:r, :c-1] 

    # diag down-right
    empty_arr[1:, 1:] += arr[:r-1, :c-1] 

    # rewrite as 1 where >= 3, else 0 -> overwrite
    arr = np.where(empty_arr >= 3 , 1, 0)

Total shaded in generation 400: 107469
Total time: 0.78
Total shaded in generation 800: 428269
Total time: 7.6
Total shaded in generation 1200: 962401
Total time: 2.6e+01
Total shaded in generation 1600: 1709869
Total time: 6.1e+01
Total shaded in generation 2000: 2670669
Total time: 1.2e+02


In [7]:
for i in [10,100,200, 500,1000,1500, 2000]:
    print(sum_n[i-1] / (i**2))

0.89
0.6869
0.676725
0.670676
0.668669
0.6680004444444444
0.66766725
